In [1]:
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn
import time
import matplotlib.pyplot as plt
import matplotlib

import preprocessing
import train
import models

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
my_lstm = models.Lstm(18, 64, 1,'../data/M_data_2019_04_16','train climateschlafzimmer')

In [5]:
# validate train data
print(my_lstm.train_set.shape)
#print(train_set[0, 0, 0], train_set[-1, -1, 0])
# validate test
print(my_lstm.test_set.shape)
#print(test_set[0, 0, 0], test_set[-1, -1, 0])
print(my_lstm.val_set.shape)

AttributeError: 'Lstm' object has no attribute 'train_set'

In [18]:
my_lstm.train_set[0,:,:].shape

(1439, 18)

In [19]:
my_lstm.y_train.shape

AttributeError: 'Lstm' object has no attribute 'y_train'

In [6]:
samples_per_day = 320#1440
learning_rate = 0.005

n_epochs = 10
batch_size = 128

In [9]:
X_train, y_train = preprocessing.to_supervised(train_set,batch_size,samples_per_day)

NameError: name 'train_set' is not defined

In [37]:
X_val, y_val = to_supervised(val_set,batch_size,samples_per_day)

In [38]:
X_test, y_test = to_supervised(test_set,batch_size,samples_per_day)

In [39]:
X_train.shape

(13750, 320, 17)

In [40]:
y_train.shape

(13750, 320)

In [41]:
X = torch.from_numpy(X_train).type(torch.float32)

In [42]:
X

tensor([[[1.9000e+01, 3.4000e+00, 2.0000e+00,  ..., 2.0190e+03,
          1.6667e-02, 1.0000e+00],
         [1.9000e+01, 3.4000e+00, 2.0000e+00,  ..., 2.0190e+03,
          3.3333e-02, 1.0000e+00],
         [1.9000e+01, 3.4000e+00, 2.0000e+00,  ..., 2.0190e+03,
          5.0000e-02, 1.0000e+00],
         ...,
         [1.9000e+01, 4.4000e+00, 2.0000e+00,  ..., 2.0190e+03,
          5.3000e+00, 1.0000e+00],
         [1.9000e+01, 4.4000e+00, 2.0000e+00,  ..., 2.0190e+03,
          5.3167e+00, 1.0000e+00],
         [1.9000e+01, 4.4000e+00, 2.0000e+00,  ..., 2.0190e+03,
          5.3333e+00, 1.0000e+00]],

        [[1.9000e+01, 3.4000e+00, 2.0000e+00,  ..., 2.0190e+03,
          3.3333e-02, 1.0000e+00],
         [1.9000e+01, 3.4000e+00, 2.0000e+00,  ..., 2.0190e+03,
          5.0000e-02, 1.0000e+00],
         [1.9000e+01, 3.2000e+00, 2.0000e+00,  ..., 2.0190e+03,
          6.6667e-02, 1.0000e+00],
         ...,
         [1.9000e+01, 4.4000e+00, 2.0000e+00,  ..., 2.0190e+03,
          5.316

In [43]:
X_v = torch.from_numpy(X_val).type(torch.float32)

In [44]:
X_t = torch.from_numpy(X_test).type(torch.float32)

In [45]:
print(X.shape, X_val.shape , X_test.shape)

torch.Size([13750, 320, 17]) (799, 320, 17) (799, 320, 17)


In [46]:
# normalize
#X = X / float(X_train.shape[0])

In [47]:
y = torch.from_numpy(y_train).type(torch.float32)

In [48]:
y_v = torch.from_numpy(y_val).type(torch.float32)

In [49]:
y_t = torch.from_numpy(y_test).type(torch.float32)

In [50]:
y.shape

torch.Size([13750, 320])

In [51]:
X.shape[2]

17

In [54]:
features

17

In [53]:
# Regularisierung
weight_decay=0.0

# the model
hidden_dim = 64
features = X.shape[2]
#model = Lstm(features, hidden_dim, d_out=1)
N = X_train.shape[0]

In [517]:
# ADAM
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [518]:
# Loss
criterion = nn.MSELoss()

In [519]:
loss_hist = []
outputs= []

In [ ]:
# Train
epochs = range(n_epochs)
idx = 0

for t in epochs:
    start = time.time()      
    #or batch in range(0, int(N/batch_size)+1):
    #   if(batch<int(N/batch_size)):
    #   # Step 1. Calculate Batch
    #       batch_x = X[batch * batch_size : (batch + 1) * batch_size, :,:]  
    #       # convert to: sequence x batch_size x n_features 
    #       #batch_x = batch_x.reshape(batch_size, samples_per_day, features)#.transpose(0,1)  
    #       batch_y = y[batch * batch_size : (batch + 1) * batch_size] 
    #       
    #       #print(X.shape, batch_x.shape, batch_y.shape)
    #   else:
    #       batch_x = X[(batch - 1) * batch_size +(N % batch_size): batch * batch_size + (N % batch_size), :]   
    #       # convert to: sequence x batch_size x n_features 
    #       #batch_x = batch_x.reshape(batch_size, samples_per_day, features).transpose(0,1)        
    #       batch_y = y[(batch - 1) * batch_size + (N % batch_size): (batch + 1) * batch_size + (N % batch_size)]    # Step 2. Remember that Pytorch accumulates gradients.
    
    # We need to clear them out before each instance
    model.zero_grad()
        
    # Also, we need to clear out the hidden state of the LSTM,
    # detaching it from its history on the last instance.
    model.hidden = model.init_hidden(batch_size)
    
    # Step 3. Run our forward pass.
    output = model(X)
    #outputs.append(output)
    # Step 4. Berechne den Fehler mit dem letzten output 
    loss = criterion(output[-1,:,-1], batch_y[-1,:])
    #print(output.shape, batch_y.shape)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Berechne den Fehler (Ausgabe des Fehlers alle 100 Iterationen)
    if t % 1 == 0:
        loss_hist.append(loss.item())
        print(t, loss.item(), time.time()-start)


In [ ]:
plt.plot(loss_hist);

In [ ]:
outputs

In [ ]:

new_val = []
for items in range(output.shape[0]):
    dff = batch_y.numpy()
    new_val.append(output[items,:,-1].detach().mean())
    #print(dff[items])
    #np.append(dff,new_val)
    #plt.plot(output[items,:].detach().numpy());
    #plt.plot(dff);
    plt.figure(1, figsize=(16, 300))
    plt.subplot(output.shape[0]//2,2,items+1)
    plt.plot(dff[items])
    plt.plot(output[items,:,-1].detach().numpy());
    plt.legend(['real','predicted']);

In [ ]:
    plt.plot(outputs);
   # plt.plot(batch_y.numpy()[1]);

In [ ]:
plt.plot(output[-1,:,-1].detach().numpy());
plt.plot(batch_y[-1,:].numpy());

In [ ]:
batch_y[-1,:]

In [ ]:
output.shape

In [257]:
batch_y

tensor([[18.6000]])

In [ ]:
output[-1,:,-1].detach().mean()

In [ ]:
output_test = model(X_Test)

In [ ]:
evaluate_forecasts(y_test, output_test)